In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import folium
from folium.plugins import FastMarkerCluster

In [2]:
datum = "EPSG:4326"
long = -79.2241
lat = 43.4305
earth_circumference = 40074 #km
lat_radians = lat * (np.pi/180) #conversion of degrees to radians
long_km = 360 / (earth_circumference * np.cos(lat_radians)) 
lat_km = 1/110.574 
n=10
shift = 2 
portion = 1


In [3]:
#Constant function of '1s' at every spatial location. Homogeneous.
def ones_2d(size=n):
    return np.ones((n,n))

# Random normally distributed values, can set the mean and standard deviation. Can be scaled with the parameter A.
def normal_dist_2d(mean=0,sd=1,size=n,A=1):
    return A*np.random.normal(mean,sd, size =(n,n))

# https://en.wikipedia.org/wiki/Gaussian_function#Two-dimensional_Gaussian_function  
def gaus2d(x=0, y=0, mx=long, my=lat, sx=0.009, sy=0.009,shift_x=0.01,shift_y=0.00):
    # Different amplitude parameters, wikipedia uses A=1, however the chosen amplitude is usually selected
      # for gaussian normalization (integral = 1)
    A = 1. / (2 * np.pi*sx * sy)
    
    return A * np.exp(-((x - mx - shift_x)**2. / (2. * sx**2.) + (y - my - shift_y)**2. / (2. * sy**2.)))


# Canadian Cities
https://simplemaps.com/data/canada-cities

Data updated as of August 26, 2020

Only includes cities with >1000 individuals.

In [4]:
canadian_cities = pd.read_csv(r"C:\Users\jreye\Desktop\NS_DATA\simplemaps_canadacities_basic\canadacities.csv")
len(canadian_cities)

1738

In [5]:
geometry = [Point(xy) for xy in zip(canadian_cities["lng"],canadian_cities["lat"])]
gdf = gpd.GeoDataFrame(canadian_cities,crs=datum,geometry=geometry)

In [6]:
gdf.province_id.unique()

array(['ON', 'QC', 'BC', 'AB', 'MB', 'NS', 'SK', 'NL', 'NB', 'PE', 'YT',
       'NT', 'NU'], dtype=object)

In [7]:
d = {}
for city in gdf.province_id.unique():
    d["city_{}".format(city)] = gdf.loc[gdf["province_id"]==city]

In [28]:
provinces = list(d.keys())
provinces

['city_ON',
 'city_QC',
 'city_BC',
 'city_AB',
 'city_MB',
 'city_NS',
 'city_SK',
 'city_NL',
 'city_NB',
 'city_PE',
 'city_YT',
 'city_NT',
 'city_NU']

In [9]:
d.get("city_ON")

,city,city_ascii,province_id,province_name,lat,lng,population,density,timezone,ranking,postal,id,geometry
0,Toronto,Toronto,ON,Ontario,43.7417,-79.3733,5429524.0,4334.4,America/Toronto,1,M5T M5V M5P M5S M5R M5E M5G M5A M5C M5B M5M M5...,1124279679,POINT (-79.37330 43.74170)
5,Ottawa,Ottawa,ON,Ontario,45.4247,-75.6950,989567.0,334.0,America/Montreal,1,K4P K4M K4A K4B K4C K7S K1S K1R K1P K1W K1V K1...,1124399363,POINT (-75.69500 45.42470)
6,Mississauga,Mississauga,ON,Ontario,43.6000,-79.6500,721599.0,2467.6,America/Toronto,2,L4W L4V L4T L4Z L4Y L4X L5R L5V L5W L5A L5B L5...,1124112672,POINT (-79.65000 43.60000)
9,Hamilton,Hamilton,ON,Ontario,43.2567,-79.8692,693645.0,480.6,America/Toronto,2,L0R L0P L8W L8V L8T L8S L8R L8P L8G L8E L8N L8...,1124567288,POINT (-79.86920 43.25670)
10,Brampton,Brampton,ON,Ontario,43.6833,-79.7667,593638.0,2228.7,America/Toronto,2,L7A L6T L6W L6V L6P L6S L6R L6Y L6X L6Z,1124625989,POINT (-79.76670 43.68330)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,Laird,Laird,ON,Ontario,46.3833,-84.0667,1047.0,10.2,America/Detroit,4,P0S,1124000662,POINT (-84.06670 46.38330)
1715,Plantagenet,Plantagenet,ON,Ontario,45.5321,-74.9956,1027.0,260.5,America/Toronto,3,K0B,1124496473,POINT (-74.99560 45.53210)
1727,Papineau-Cameron,Papineau-Cameron,ON,Ontario,46.3000,-78.7333,1016.0,1.8,America/Toronto,4,P0H,1124000867,POINT (-78.73330 46.30000)
1728,Assiginack,Assiginack,ON,Ontario,45.7333,-81.8000,1013.0,4.5,America/Toronto,3,P0P,1124000091,POINT (-81.80000 45.73330)


In [10]:
#mean for ontario
mean_lat_on = np.mean(d["city_ON"]["lat"])
mean_lng_on = np.mean(d["city_ON"]["lng"])

#mean for canada
gdf_mean_lat = np.mean(gdf.lat)
gdf_mean_lng = np.mean(gdf.lng)

In [11]:
my_map = folium.Map(location=[gdf_mean_lat,gdf_mean_lng], zoom_start=5)
#folium.GeoJson(data = gdf).add_to(my_map)
my_map.add_child(FastMarkerCluster(gdf[["lat","lng"]].values.tolist()))
my_map

In [13]:
my_map = folium.Map(location=[mean_lat_on,mean_lng_on], zoom_start=5)

folium.GeoJson(data = d["city_ON"],popup=folium.GeoJsonPopup(fields=["city",
                                                                     "province_id",
                                                                     "lat",
                                                                     "lng",
                                                                     "population",
                                                                     "density"])).add_to(my_map)

folium.GeoJson(data = d["city_NS"],popup=folium.GeoJsonPopup(fields=["city",
                                                                     "province_id",
                                                                     "lat",
                                                                     "lng",
                                                                     "population",
                                                                     "density"])).add_to(my_map)

my_map


# Clustering using DBSCAN

https://en.wikipedia.org/wiki/DBSCAN

https://en.wikipedia.org/wiki/Cluster_analysis#Density-based_clustering

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

https://github.com/gboeing/2014-summer-travels/blob/master/clustering-scikitlearn.ipynb

https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

https://stackoverflow.com/questions/34579213/dbscan-for-clustering-of-geographic-location-data

https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration.html#sphx-glr-auto-examples-calibration-plot-calibration-py

https://www.scikit-yb.org/en/latest/api/cluster/elbow.html#:~:text=The%20elbow%20method%20runs%20k,point%20to%20its%20assigned%20center.

In [101]:
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [301]:
canadian_cities = pd.read_csv(r"C:\Users\jreye\Desktop\NS_DATA\simplemaps_canadacities_basic\canadacities.csv")

#create geodataframe containing data with all canadian cities and a point geometry column
geometry = [Point(xy) for xy in zip(canadian_cities["lng"],canadian_cities["lat"])]
gdf = gpd.GeoDataFrame(canadian_cities,crs=datum,geometry=geometry)

In [317]:
d

{'city_ON':                   city        city_ascii province_id province_name      lat  \
 0              Toronto           Toronto          ON       Ontario  43.7417   
 5               Ottawa            Ottawa          ON       Ontario  45.4247   
 6          Mississauga       Mississauga          ON       Ontario  43.6000   
 9             Hamilton          Hamilton          ON       Ontario  43.2567   
 10            Brampton          Brampton          ON       Ontario  43.6833   
 ...                ...               ...         ...           ...      ...   
 1699             Laird             Laird          ON       Ontario  46.3833   
 1715       Plantagenet       Plantagenet          ON       Ontario  45.5321   
 1727  Papineau-Cameron  Papineau-Cameron          ON       Ontario  46.3000   
 1728        Assiginack        Assiginack          ON       Ontario  45.7333   
 1731            Prince            Prince          ON       Ontario  46.5333   
 
           lng  population 

In [302]:
#create dictionary with complete data for each province
d = {}
for city in gdf.province_id.unique():
    d["city_{}".format(city)] = gdf.loc[gdf["province_id"]==city]

#obtain province names as well as list of dictionary keys.
provinces = list(d.keys())

#obtain lat/long data for each province and the entire country as a numpy array.
d_lat_lon_numpy = {}
for province in provinces:
    d_lat_lon_numpy["{}".format(province)] = [d.get(province)[["lat","lng"]].to_numpy()]

d_lat_lon_numpy["Canada"] = [gdf[["lat","lng"]].to_numpy()]


In [303]:
d_lat_lon_numpy.keys()


dict_keys(['city_ON', 'city_QC', 'city_BC', 'city_AB', 'city_MB', 'city_NS', 'city_SK', 'city_NL', 'city_NB', 'city_PE', 'city_YT', 'city_NT', 'city_NU', 'Canada'])

# DBSCAN

In [304]:
kms_per_radian = 6371.0088
epsilon = 50 / kms_per_radian

#perform DBSCAN algorithm to each province separately as well as the entire country
for province in list(d_lat_lon_numpy.keys()):
    #Create DBSCAN object and apply to each latitude/longitude pair
    d_lat_lon_numpy["{}".format(province)].append(
        {"dbs_{}".format(province):DBSCAN(eps=epsilon, min_samples=1,algorithm = 'ball_tree',metric='haversine').fit(np.radians(d_lat_lon_numpy.get(province)[0]))})
    #Retrieve labels obtained from algorithm
    d_lat_lon_numpy["{}".format(province)].append(
        {"{}_cluster_label".format(province):d_lat_lon_numpy.get(province)[1]["dbs_{}".format(province)].labels_})
    #Obtain cluster labels
    d_lat_lon_numpy["{}".format(province)].append(
        {"{}_num_clusters".format(province):len(set(d_lat_lon_numpy.get(province)[2]["{}_cluster_label".format(province)]))})
    #
    d_lat_lon_numpy["{}".format(province)].append(
        {"{}_clusters".format(province):
        pd.Series(d_lat_lon_numpy.get(province)[0][d_lat_lon_numpy.get(province)[2]["{}_cluster_label".format(province)] == n] for n in range(d_lat_lon_numpy["{}".format(province)][3]["{}_num_clusters".format(province)]))})
    




In [305]:
d_lat_lon_numpy.get(province)[2]["{}_cluster_label".format(province)]

array([  0,   0,   1, ...,  65,   5, 189], dtype=int64)

In [306]:
d_lat_lon_numpy.get(province)[3]["{}_num_clusters".format(province)]

190

In [307]:
d_lat_lon_numpy.get(province)[4]["{}_clusters".format(province)]

0      [[43.7417, -79.3733], [45.5089, -73.5617], [45...
1      [[49.25, -123.1], [49.19, -122.8489], [48.4283...
2      [[51.05, -114.0667], [53.5344, -113.4903], [52...
3      [[49.8844, -97.1464], [49.8483, -99.95], [51.2...
4      [[44.6475, -63.5906], [45.5926, -62.6455], [45...
                             ...                        
185                               [[55.7372, -117.2014]]
186                                [[69.5554, -93.4972]]
187                               [[51.2667, -105.9667]]
188                               [[52.8284, -119.2659]]
189                               [[51.3522, -110.4739]]
Length: 190, dtype: object

## Finding the point in each cluster closest to the centroid

The objective is to determine a representative point from each cluster. 
The lat/long coordinates of the cluster's centroid is located with shapely (where lat is x and lon is y). Then we proceed to find the member of the cluster with the smallest great circle distance to the centroid.

Special note: DBSCAN clusters may be non-convex, and thus the representative point may lie outside of the cluster.

In [308]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

for province in list(d_lat_lon_numpy.keys()):

    d_lat_lon_numpy["{}".format(province)].append(
        {"{}_centermost_points".format(province):d_lat_lon_numpy.get(province)[4]["{}_clusters".format(province)].map(get_centermost_point)})

    #unzip the list of centermost points (lat,lon) tuples into separate lat/lon lists
    lats, lons = zip(*d_lat_lon_numpy.get(province)[5]["{}_centermost_points".format(province)])
    #create a pandas dataframe
    rep_points = pd.DataFrame({'lon':lons, 'lat':lats})

    d_lat_lon_numpy["{}".format(province)].append({"{}_centermost_points_numpy".format(province) : rep_points.to_numpy()})


    d_lat_lon_numpy["{}".format(province)].append(
        {"{}_gdf_cluster_samples".format(province):gpd.GeoDataFrame(rep_points, geometry=gpd.points_from_xy(rep_points.lon, rep_points.lat),crs = "EPSG:4326" )})

    


In [309]:
d_lat_lon_numpy.get(province)[5]["{}_centermost_points".format(province)]

0            (45.65, -73.3)
1      (49.1667, -123.1333)
2      (51.8236, -113.2328)
3       (50.2862, -98.8415)
4         (45.3647, -63.28)
               ...         
185    (55.7372, -117.2014)
186     (69.5554, -93.4972)
187    (51.2667, -105.9667)
188    (52.8284, -119.2659)
189    (51.3522, -110.4739)
Length: 190, dtype: object

In [311]:
d_lat_lon_numpy.get(province)[6]["{}_centermost_points_numpy".format(province)]

array([[ -73.3   ,   45.65  ],
       [-123.1333,   49.1667],
       [-113.2328,   51.8236],
       [ -98.8415,   50.2862],
       [ -63.28  ,   45.3647],
       [-106.3967,   52.6206],
       [-104.3451,   50.5166],
       [ -71.7667,   48.5   ],
       [-119.4142,   50.0833],
       [ -52.9833,   47.5167],
       [ -89.2461,   48.3822],
       [ -60.1931,   46.1389],
       [-122.7494,   53.9169],
       [ -82.95  ,   46.1833],
       [-111.3284,   57.6042],
       [-110.7783,   50.0792],
       [-119.1392,   55.1572],
       [ -79.2   ,   48.6667],
       [ -81.3333,   48.4667],
       [-105.5091,   50.4433],
       [ -77.6333,   48.4333],
       [-109.7772,   53.4236],
       [ -66.3833,   50.2167],
       [-135.0691,   60.7029],
       [-120.8476,   56.2465],
       [ -57.95  ,   48.9833],
       [-114.4053,   62.4709],
       [-115.9775,   49.6697],
       [ -61.1   ,   46.2   ],
       [-107.7939,   50.2881],
       [-128.5997,   54.5164],
       [ -94.4833,   49.7667],
       [

In [312]:
d_lat_lon_numpy.get(province)[7]["{}_gdf_cluster_samples".format(province)]

,lon,lat,geometry
0,-73.3000,45.6500,POINT (-73.30000 45.65000)
1,-123.1333,49.1667,POINT (-123.13330 49.16670)
2,-113.2328,51.8236,POINT (-113.23280 51.82360)
3,-98.8415,50.2862,POINT (-98.84150 50.28620)
4,-63.2800,45.3647,POINT (-63.28000 45.36470)
...,...,...,...
185,-117.2014,55.7372,POINT (-117.20140 55.73720)
186,-93.4972,69.5554,POINT (-93.49720 69.55540)
187,-105.9667,51.2667,POINT (-105.96670 51.26670)
188,-119.2659,52.8284,POINT (-119.26590 52.82840)


In [ ]:
my_map = folium.Map(location=[gdf_mean_lat,gdf_mean_lng], zoom_start=5)

#folium.GeoJson(data = gdf).add_to(my_map)

#my_map.add_child(FastMarkerCluster(gdf[["lat","lng"]].values.tolist()))

folium.GeoJson(data=gdf_cluster_samples).add_to(my_map) 
my_map

In [313]:
d_lat_lon_numpy["city_ON"][6]

{'city_ON_centermost_points_numpy': array([[-79.0833,  44.3167],
        [-89.2461,  48.3822],
        [-82.95  ,  46.1833],
        [-81.3333,  48.4667],
        [-94.4833,  49.7667],
        [-79.825 ,  47.7083],
        [-82.15  ,  49.35  ],
        [-92.8333,  49.7833],
        [-93.8333,  48.6333],
        [-80.6667,  48.7667],
        [-91.9167,  50.1   ],
        [-83.6544,  49.6869],
        [-86.7333,  50.    ],
        [-93.8333,  51.0333],
        [-86.3667,  48.75  ],
        [-84.7736,  47.9931],
        [-91.6167,  48.75  ],
        [-83.4   ,  47.8333],
        [-85.8333,  49.1333],
        [-88.2683,  49.0153],
        [-87.2667,  48.8167],
        [-80.6431,  51.2722],
        [-91.6667,  49.4167],
        [-94.08  ,  49.4   ]])}

In [326]:
my_map = folium.Map(location=[gdf_mean_lat,gdf_mean_lng], zoom_start=5)

for point in d_lat_lon_numpy["city_BC"][6]["city_BC_centermost_points_numpy"]  :
    loc = [point[1],point[0]]
    folium.Marker(location=loc,icon=folium.Icon(color="red")).add_to(my_map)
    #folium.Circle(radius=40000,location=[point[1],point[0]],color="red").add_to(my_map)
    
#folium.GeoJson(data = gdf).add_to(my_map)
    

my_map